In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data = pd.read_csv("C:/Users/joyh1/worldcup_predict_younghun/results.csv")
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [6]:
data.shape

(41540, 9)

In [7]:
data.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
41535,2019-11-19,Kyrgyzstan,Tajikistan,1,1,FIFA World Cup qualification,Bishkek,Kyrgyzstan,False
41536,2019-11-19,Vietnam,Thailand,0,0,FIFA World Cup qualification,Hanoi,Vietnam,False
41537,2019-11-19,Malaysia,Indonesia,2,0,FIFA World Cup qualification,Kuala Lumpur,Malaysia,False
41538,2019-11-19,Turkmenistan,Sri Lanka,2,0,FIFA World Cup qualification,Ashgabat,Turkmenistan,False
41539,2019-11-19,Lebanon,North Korea,0,0,FIFA World Cup qualification,Beirut,Lebanon,False


In [10]:
data.isnull().sum()

date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [12]:
type(data['home_score'][0]) # score칼럼의 형태는 정수형이므로 밑에서 조건문을 달 때 대소비교(<,>)를 할 수 있을 것이다.

numpy.int64

In [13]:
# 홈스코어 어웨이 스코어를 비교해서 승/패자 값을 가린 후 winning_team이라는 새로운 칼럼에 추가하기

winner = [] # 빈리스트를 설정하고 값을 채운후 data라는 dataframe에 새로운 칼럼으로 추가할 예정
for i in range(len(data['date'])):
    if data['home_score'][i] > data['away_score'][i]:
        winner.append(data['home_team'][i])
    elif data['home_score'][i] < data['away_score'][i]:
        winner.append(data['away_team'][i])
    else:
        winner.append("Tie")
        
data['winning_team'] = winner

data.head()                # 원하는 대로 '승팀' 칼럼을 추가

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winning_team
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,Tie
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,Tie
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,Scotland
